<center> <h1> HyIPO: Hyped Initial Product Offerings </h1> </center>
<center> <h2> Notebook:  Data Acquisition and Feature Engineering </h2> </center>
<p><strong>Goal:</strong> feature engineering and extraction

In this notebook, I used Pytrends, an unofficial Google Trends API that provides different methods to download reports of trending results from google trends. I managed to download for each issuer in the list their search data from google trends API (https://trends.google.com/) in order to assert potential investors’ appetite for each IPO; I used the number of spikes in popularity (observations > mean) during the last 2 weeks before the IPO date. 


## Loading Packages

In [ ]:
pip install pytrends

  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=93857bfc48c845e5dd5c9ef7a030befefce1abd84091cf26b8a84fba3b272d3d
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


In [ ]:
# Import all the necessary packages
import pandas as pd
import numpy as np

import statsmodels.api as sm
import scipy.stats as st

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import statistics
import warnings
warnings.filterwarnings('ignore')

from google.colab import files


from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, classification_report, f1_score, accuracy_score,r2_score)
from sklearn.metrics import precision_recall_fscore_support, mutual_info_score
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

import datetime
from pandas.tseries.offsets import *

import requests
from bs4 import BeautifulSoup
import re # a library to do regular expression matching

from pytrends.request import TrendReq


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Necessary Functions for this Notebook

In [ ]:
# FUNCTION --> # Shows you if the input date is on a business day or not
from pandas.tseries.offsets import BDay

bdays=BDay()

def is_business_day(date):
  return date == date + 0*bdays

In [ ]:
# special character removal
def remove_special_characters(text):
    # pattern = r'[^0-9\s]+' if not remove_digits else r'[^a-zA-Z\s]'
    pattern = r'[0-9]'
    text = re.sub('[!,*)=@#%(&$_?.^]', '', text) 
    text = re.sub(pattern, '', text) 
    return text 

## Data Loading

### Intermediary Dataset

In [ ]:
!gdown --id 1T98B_9XRif7RPHuZYWMc6H9yLPIkIUcP

Downloading...
From: https://drive.google.com/uc?id=1T98B_9XRif7RPHuZYWMc6H9yLPIkIUcP
To: /content/final_data_with_features.csv
100% 2.21M/2.21M [00:00<00:00, 201MB/s]


In [ ]:
df = pd.read_csv('/content/final_data_with_features.csv')

In [ ]:
df = df.drop(axis=1, columns='Unnamed: 0')

In [ ]:
df.head()

,Trade Date,Issuer,Symbol,Lead/Joint-Lead Managers,Offer Price,Opening Price,1st Day Close,1st Day % Px Chng,$ Change Opening,$ Change Close,...,priceVIX1mc,priceVIX3ms,priceVIX3mc,VIX 1 Week % Px Chng,VIX 1 Month % Px Chng,VIX 3 Months % Px Chng,Sentiment_survey,Top IB,weekday,tyield
0,2020-01-17 00:00:00,I-Mab,IMAB,Jefferies/ CICC,14.0,14.75,12.75,-0.089286,0.75,-1.25,...,12.10,15.20,12.10,-0.100372,-0.182432,-0.203947,0.143266,0,4,0.01631
1,2020-01-17 00:00:00,LIZHI,LIZI,Credit Suisse/ Citigroup,11.0,11.03,11.63,0.057273,0.03,0.63,...,12.10,15.20,12.10,-0.100372,-0.182432,-0.203947,0.143266,1,4,0.01631
2,2020-01-17 00:00:00,Phoenix Tree Holdings Ltd.,DNK,Citigroup/ Credit Suisse/ J.P. Morgan,13.5,13.50,13.50,0.000000,0.00,0.00,...,12.10,15.20,12.10,-0.100372,-0.182432,-0.203947,0.143266,1,4,0.01631
3,2020-01-17 00:00:00,Velocity Financial,VEL,Wells Fargo Securities/ Citigroup/ JMP Securities,13.0,13.85,13.51,0.039231,0.85,0.51,...,12.10,15.20,12.10,-0.100372,-0.182432,-0.203947,0.143266,1,4,0.01631
4,2020-01-24 00:00:00,Gores Holdings IV,GHIVU,Deutsche Bank Securities,10.0,10.52,10.45,0.045000,0.52,0.45,...,14.56,14.67,14.56,0.175141,-0.071429,-0.007498,0.208334,0,4,0.01501


In [ ]:
pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=2, backoff_factor=0.1, requests_args={'verify':False})
# pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), proxies=['https://85.25.91.156:5566'], retries=2, backoff_factor=0.1, requests_args={'verify':False})
#pytrends = TrendReq(hl='en-US', tz=360)

In [ ]:
stocks = df['Issuer'].to_list()
start_date_y = df['year'].to_list()
start_date_m = df['month'].to_list()
start_date_d = df['day'].to_list()

In [ ]:
from pandas.tseries.offsets import *

m1 = []
m2 = []
c = []
std = []

for i in range(0, len(stocks)):
  end = datetime.datetime(start_date_y[i], start_date_m[i], start_date_d[i])  
  end = end - datetime.timedelta(days=1)
  if is_business_day(end) == False:
    end = end - datetime.timedelta(days=2)
  start = (end - datetime.timedelta(days=14))
  start = start.date()
  s = datetime.datetime(start.year, start.month, start.day)  
  tf = str(s.year)+'-'+str(s.month)+'-'+str(s.day)+ ' '+str(end.year)+'-'+str(end.month)+'-'+str(end.day)
 
  text = stocks[i]
  special_char_pattern = re.compile(r'([{.()!}])')
  text = special_char_pattern.sub(" \\1 ", text)
  text = remove_special_characters(text)  
  text = re.sub(' +', ' ', text) 
  text = text.strip()
  text = text
  kw_list = [text]  
 
  mean = 0
  median = 0 
  count = 0 
  st = 0

  try:
    pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=2, backoff_factor=0.1, requests_args={'verify':False})
    pytrends.build_payload(kw_list, cat=0, timeframe=tf , geo='', gprop='')
    interest_over_time_df = pytrends.interest_over_time()
    st = interest_over_time_df[text].std()
    mean = interest_over_time_df[text].mean()
    median = interest_over_time_df[text].median()
    count = interest_over_time_df[text][interest_over_time_df[text]>mean].count()

  except:
    print('error')  
  m1.append(mean)
  m2.append(median)
  c.append(count)
  std.append(st)


error
error


In [ ]:
df['Googlemean'] = m1
df['Googlemedian'] = m2
df['Googlecount'] = c
df['Google std'] = std

## Exporting the new data set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df.to_csv('/content/drive/My Drive/IPO_Project/data/processed_data/data_features_GT.csv') 